In [1]:
include("./param.jl");
include("./aux_functions.jl");

using Main.aux_functions
using Main.param
using ADCME
using SparseArrays

using DelimitedFiles
using Dates
using Plots



2021-05-30 19:53:49.424382: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library cudart64_100.dll


In [2]:
maxiter = 10
N_steps = 3


3

In [3]:
print("maximum iteration: ",maxiter,"\t","Number of steps: ",N_steps,"\n")
param_model_val = param_model(N_steps=N_steps);
tf_variables, h_t, q_t_x, q_t_y = Darcy_flow_solver(param_model_val);

loss,dw_2, opt_ADAM, opt_LFGS, opt_ADAM_sum, opt_LFGS_sum, diff_eval,p_pre_soft_max, p = Info_upscale(tf_variables,param_model_val,q_t_x, q_t_y,maxiter)

sess = Session(); init(sess);


global N_k_dis_ = 4
global T_exp = -2
T_=  10.0 .^ -T_exp


maximum iteration: 10	Number of steps: 3
2.830469177224561e-5C:\Users\dbeli\.julia\adcme\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\dbeli\.julia\adcme\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\dbeli\.julia\adcme\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\dbeli\.julia\adcme\lib\site-packa

100.0

In [4]:

# save_values(sess,param_model_val,tf_variables,q_t_x, q_t_y,p,"w")

print_status(sess,loss,diff_eval,T_exp,T_,N_k_dis_,tf_variables)
# ScipyOptimizerMinimize(sess, opt_LFGS_sum,feed_dict = Dict(tf_variables.lambda => ones(1)*T_,tf_variables.N_k_dis=>8))
BFGS!(sess,dw_2*1e5,options=Dict("maxiter"=> maxiter, "ftol"=>1e-12, "gtol"=>1e-12))

# save_values(sess,param_model_val,tf_variables,q_t_x, q_t_y,p,"a")
print_status(sess,loss,diff_eval,T_exp,T_,N_k_dis_,tf_variables)

check_diff = run(sess,diff_eval,feed_dict = Dict(tf_variables.lambda => ones(1)*T_,tf_variables.N_k_dis=>N_k_dis_))



19:55-2	100.0	0.0007267508285325198	5.5210782319984135		4
C:\Users\dbeli\.julia\adcme\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
┌ Info: Optimization starts...
└ @ ADCME C:\Users\dbeli\.julia\packages\ADCME\cupNK\src\optim.jl:332
iter 0, current loss=0.0008692784609354104
iter 1, current loss=0.0007249955080323163
================ STEP 0 ===============
iter 2, current loss=0.0002814875674504895
================ STEP 1 ===============
iter 3, current loss=0.00014080786291257642
================ STEP 2 ===============
iter 4, current loss=5.908145140680884e-5
================ STEP 3 ===============
iter 5, current loss=2.5844710307020072e-5
================ STEP 4 ===============
iter 6, current loss=1.2100804233498268e-5
================ STEP 5 ===============
i

3.70757690127924e-5

In [5]:
T_exp_final =4

while T_exp <= T_exp_final

    global T_=  10.0 ^ (-T_exp) 

    print_status(sess,loss,diff_eval,T_exp,T_,N_k_dis_,tf_variables)
    check_diff_ = run(sess,diff_eval,feed_dict = Dict(tf_variables.lambda => ones(1)*T_,tf_variables.N_k_dis=>N_k_dis_))  
    print_status(sess,loss,diff_eval,T_exp,T_,N_k_dis_,tf_variables)
    # ScipyOptimizerMinimize(sess, opt_LFGS,feed_dict = Dict(tf_variables.lambda => ones(1)*T_,tf_variables.N_k_dis=>N_k_dis_))
    BFGS!(sess,loss*1e5,options=Dict("maxiter"=> 100, "ftol"=>1e-12, "gtol"=>1e-12),feed_dict = Dict(tf_variables.lambda => ones(1)*T_,tf_variables.N_k_dis=>N_k_dis_))

    print_status(sess,loss,diff_eval,T_exp,T_,N_k_dis_,tf_variables)
    if round(T_exp,digits=2)%1 == 0

        # save_values(sess,param_model_val,tf_variables,q_t_x, q_t_y,p)
        check_diff_ = run(sess,diff_eval,feed_dict = Dict(tf_variables.lambda => ones(1)*T_,tf_variables.N_k_dis=>N_k_dis_))  
        global N_k_dis_ = update_K_p(sess,param_model_val,tf_variables,check_diff_,N_k_dis_,p_pre_soft_max)

    end

    global T_exp += 0.1
end

19:56-2	100.0	3.70757690127924e-5	0.21644049768110213		4
19:56-2	100.0	3.70757690127924e-5	0.21644049768110213		4
C:\Users\dbeli\.julia\adcme\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
┌ Info: Optimization starts...
└ @ ADCME C:\Users\dbeli\.julia\packages\ADCME\cupNK\src\optim.jl:332
iter 0, current loss=21644.049768088513
iter 1, current loss=11691.525324682636
================ STEP 0 ===============
iter 2, current loss=8105.697529602203
================ STEP 1 ===============
iter 3, current loss=7533.509126692936
================ STEP 2 ===============
iter 4, current loss=7190.783788108559
================ STEP 3 ===============
iter 5, current loss=6942.582274202099
================ STEP 4 ===============
iter 6, current loss=6447.126407047822
===========